Kaggle Competition for SF Crime Stats using XGB Multi-Class Classification

In [10]:
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [11]:
# Data from Kaggle
train = pd.read_csv(r'C:\Career\LighthouseLabs\DataScience_Week9\Kaggle_XGBoost_Multi-Class_Crime\train.csv')
test = pd.read_csv(r'C:\Career\LighthouseLabs\DataScience_Week9\Kaggle_XGBoost_Multi-Class_Crime\test.csv')

In [4]:
test.shape

(884262, 7)

In [5]:
train.shape

(878049, 9)

In [6]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [7]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


Data Pre-processing

In [17]:
# Make lowercase for ease of typing
train.columns = [item.lower() for item in train.columns]

In [ ]:
# Simplify 'date' to single column
train['Dates'] = pd.to_datetime(train['Dates'])

train['year'] = train['Dates'].dt.year
train['month'] = train['Dates'].dt.month
train['day_of_month'] = train['Dates'].dt.day
train['day_of_week'] = train['Dates'].dt.day_of_week
train['hour'] = train['Dates'].dt.hour
train.drop(['Dates', 'DayOfWeek'], axis = 1, inplace = True)

In [18]:
top200Des = train['descript'].value_counts()[:200].index
train['descript'] = train['descript'].apply(lambda x :x if x in top200Des else 'other' )
x = train.drop(['category', 'address'], axis = 1)
y = train['category']

In [19]:
le = LabelEncoder()
y = le.fit_transform(y)
x['descript'] = le.fit_transform(x['descript'])
x['pddistrict'] = le.fit_transform(x['pddistrict'])
x['resolution'] = le.fit_transform(x['resolution'])

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)
print('x train :', x_train.shape,'\t\tx test :', x_test.shape)
print('y train :', y_train.shape,'\t\ty test :', y_test.shape)

x train : (702439, 10) 		x test : (175610, 10)
y train : (702439,) 		y test : (175610,)


In [21]:
# Scale the features so their range doesn't distort their relative importance
mms = MinMaxScaler()
x_train = mms.fit_transform(x_train)
x_test = mms.transform(x_test)

In [22]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

Time to train the model.

In [23]:
# K Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

In [24]:
# KNN Accuracy
accuracy_score(y_pred, y_test)

0.4572575593645009

In [25]:
# Random Forest
rfc = RandomForestClassifier(random_state = 42)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

In [26]:
# Random Forest Accuracy
accuracy_score(y_pred, y_test)

0.9361938386196685

In [27]:
# Decision Tree
dtc = DecisionTreeClassifier(random_state= 1)
dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)

In [28]:
# Decision Tree Accuracy
accuracy_score(y_pred, y_test)

0.9544843687717101

In [ ]:
# XGBoost
model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
# XGBoost Accuracy
accuracy_score(y_pred, y_test)

Highest Accuracy came from Decision Tree model at 0.9544843687717101.

In [ ]:
# Submission method using in Kaggle Housing XGBoost Regression
output = pd.DataFrame({"Id":test_data.index, "category":predictions})
output.to_csv('submission.csv', index=False)

In [8]:
# X_train = train.copy()
# X_train = X_train.drop(['Category'], axis=1)
# y_train = train['Category']
# X_test  = test.copy()

In [9]:
# simple preprocessing: imputation; substitute any 'NaN' with mean value
#===========================================================================
X_train = X_train.fillna(X_train.mean())
X_test  = X_test.fillna(X_test.mean())

C:\Users\User\AppData\Local\Temp\ipykernel_16580\3498501342.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


KeyboardInterrupt: 

In [ ]:
y_train.shape

In [ ]:
# write out CSV submission file
#===========================================================================
output = pd.DataFrame({"Id":test_data.index, 'Category':predictions})
output.to_csv('submission.csv', index=False)